<a href="https://colab.research.google.com/github/ishikawa10/kyotodata/blob/master/Transformer_by_tf.keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!git clone --recursive https://github.com/iki-taichi/tf-keras-transformer.git

fatal: destination path 'tf-keras-transformer' already exists and is not an empty directory.


In [5]:
!git clone --recursive https://github.com/ishikawa10/kyotodata.git

Cloning into 'kyotodata'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [0]:
!mkdir tf-keras-transformer/data/kyoto_corpus
!cat kyotodata/* > tf-keras-transformer/data/kyoto_corpus/wiki_corpus_2.01.tar.gz

In [9]:
%%bash 
sed -i -e "s/urlretrieve(/#urlretrieve(/" tf-keras-transformer/src/get_kyoto_corpus.py
grep -n urlretrieve tf-keras-transformer/src/get_kyoto_corpus.py

16:from urllib.request import urlretrieve
35:    ##urlretrieve(DOWNLOAD_URL, file_path)


In [10]:
!pip install sentencepiece

In [0]:
import os
os.chdir('tf-keras-transformer')

In [12]:
!python src/get_kyoto_corpus.py

start downloading...
downloaded
start editing files...
to convert 14111 xml files...
data sample:(data/kyoto_corpus/GNM/GNM00340.xml)['Yoshino-onsen Hot Spring is located in Yoshino-cho, Yoshino County, Nara Prefecture.', '吉野温泉（よしのおんせん）は、奈良県吉野郡吉野町にある温泉。']
data sample:(data/kyoto_corpus/GNM/GNM00340.xml)['Train: Get off at Kintetsu Yoshino Line Yoshino Station (Nara Prefecture).', '鉄道 近鉄吉野線吉野駅 (奈良県)下車。']
data sample:(data/kyoto_corpus/GNM/GNM00340.xml)['Iron carbonate and hydrogen carbonate spring', '炭酸鉄泉、炭酸水素塩泉']
data sample:(data/kyoto_corpus/GNM/GNM00340.xml)['The source of the hot spring is located near the Kissuiin in Mt. Yoshino.', '源泉は吉野山中の吉水院の近くに湧出している。']
data/kyoto_corpus/GNM/GNM00155.xml skipped because of not well-formed (invalid token): line 2811, column 2425
data/kyoto_en_ja_valid.csv,data/kyoto_en_ja.csv were created


In [0]:
from src.fitting import FitEnvironment

In [0]:
env = FitEnvironment(
        use_tpu=True,
        batch_size=8,
        input_len=(1024, 1024),
        num_epoch=5,
        output_dir='/content/drive/My Drive/transformer_model',
        data_path=['data/kyoto_en_ja.csv'],
        valid_path=['data/kyoto_en_ja_valid.csv'],
        resume_model_path=None,
        resume_initial_epoch=None,
    )

In [15]:
wrapper = env.run()

INFO:tensorflow:training starting with env={
  "model_config": "{\n  \"use_tpu\": true,\n  \"tpu_grpc_url\": \"grpc://10.65.70.50:8470\",\n  \"src_tokenizer\": \"sp_uncase_en_ja_40000\",\n  \"tar_tokenizer\": \"sp_uncase_en_ja_40000\",\n  \"use_same_embed\": true,\n  \"block_num\": [\n    6,\n    6\n  ],\n  \"embed_dim\": 768,\n  \"hidden_dim\": 3072,\n  \"head_num\": 12,\n  \"attention_activation\": \"relu\",\n  \"feed_forward_activation\": \"gelu\",\n  \"dropout_rate\": 0.1,\n  \"input_len\": [\n    1024,\n    1024\n  ],\n  \"token_num\": [\n    null,\n    null\n  ]\n}",
  "use_tpu": true,
  "input_len": [
    1024,
    1024
  ],
  "work_dir": "model/transformer",
  "output_dir": "/content/drive/My Drive/transformer_model",
  "data_path": [
    "data/kyoto_en_ja.csv"
  ],
  "valid_path": [
    "data/kyoto_en_ja_valid.csv"
  ],
  "show_model_summary": true,
  "batch_size": 8,
  "num_epoch": 5,
  "warm_step": 4000,
  "train_callbacks": [
    "<src.custom_callbacks.BatchLearningRateSche

In [0]:
tf.keras.backend.clear_session()
from src.transformer import TransformerWrapper
trans = TransformerWrapper('/content/drive/My Drive/transformer_model/model.05.hdf5')

In [21]:
trans('Kyoto is a Japanese city.')

'京都(きょうと)は、日本の京都市のひとつ。'

In [26]:
trans('建立')

'用語'

In [7]:
!ls

drive  kyotodata  sample_data  tf-keras-transformer


In [0]:
import google.colab
import googleapiclient.discovery
import googleapiclient.http

In [0]:
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

In [0]:
saving_filename = "data/kyoto_en_ja_valid.csv"

file_metadata = {
  'name': saving_filename,
  'mimeType': 'application/octet-stream'
}
media = googleapiclient.http.MediaFileUpload(saving_filename, 
                        mimetype='application/octet-stream',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()

In [3]:
!ls '/content/drive/My Drive'

transformer_model


In [16]:
import tensorflow as tf
print(tf.__version__)

import keras
print(keras.__version__)

1.13.1
2.2.5


In [0]:
!apt-get --purge remove cuda*

In [0]:
!apt autoremove

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb

In [0]:
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb

In [0]:
!apt-key add /var/cuda-repo-*/7fa2af80.pub

In [0]:
!apt-get update

In [0]:
!apt-get install cuda-9-0

In [0]:
!pip uninstall -y tensorflow

In [0]:
!pip install tensorflow==1.13.1

In [0]:
import tensorflow as tf
print(tf.__version__)